Оригинальный датасет https://www.kaggle.com/imakash3011/customer-personality-analysis/version/1

На его примере рассматривается анализ клиентской базыдля поиска идеальных клиентов и прогнозирования вероятности покупки.

# Чтение данных

Импортируем библиотеки

``` python
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

```


Читаем файл

``` python
df=pd.read_excel('marketing_campaign.xlsx')
df
```


Информация о таблице

```python
df.info()
```

# Исследуем данные

Описание полей:
- 'Year_Birth' дата рождения
- 'Education' образование
- 'Marital_Status' семейный статус
- 'Income' годовой доход семьи
- 'Dt_Customer' дата регистрации клиента на сайте компании 
- 'Recency' количество дней с момента последней покупки
- 'NumWebVisitsMonth' количество визитов на сайт в месяц
- 'Complain' наличие жалоб
- 'Response' отклик на предложение (1 - если принял)
- 'Total_Engaged_Days' сколько всего дней вовлечен в коммуникацию
- 'Age' - лет с момента регистрации



```python
df.describe()
```

График, визуализирует распределение данных.


```python
fig, axes = plt.subplots(2, 1, figsize=(10,5))

sns.boxplot(data = df, x = 'Income', color = 'red', ax = axes[0])
sns.histplot(data = df, x = 'Income', kde=True, color='blue', ax = axes[1])

plt.show()
```

Убираем выбросы.

```python
df=df[df['Income']<120000]
df=df[df['NumWebVisitsMonth']<13]
df=df[df['Year_Birth']>1940]
```

Показывает количество уникальных значений.

```python
df.nunique()
```

Частота встречаемости значения.

```python
df['Marital_Status'].value_counts()
```

Кодирование переменных.

```python
voc_status={'Single':0, 'Together':1, 'Married':2, 'Divorced':3, 'Widow':3, 'Alone':3, 'Absurd':4, 'YOLO':4}
voc_ed={'Basic':0, 'Graduation':1, 'Master':2, '2n Cycle':3, 'PhD':4}
df['Marital_Status_Code']=df['Marital_Status'].map(voc_status)
df['Education_Code']=df['Education'].map(voc_ed)
```

Визуализация переменных.

```python
to_plot = ['Income', 'Year_Birth',  'Recency', 'NumWebVisitsMonth', 'Response',
       'Total_Engaged_Days', 'Age', 'Education_Code', 'Marital_Status_Code', 'Complain']

sns.pairplot(df[to_plot], hue = 'Response')
plt.show()
```

Укрупненный график рассенияния.

```python
sns.jointplot(x='Recency', y='Income', data=df, kind='scatter');
```

Тепловая карта корреляции.

```python
fig = plt.figure(figsize=(8,6))
plt.title('Корреляция между переменными')
sns.heatmap(df.corr(), annot=True, cmap="Greens");
```

# Кластеризация

```python
from sklearn.cluster import KMeans
from sklearn import preprocessing
col=['Income', 'Education_Code', 'Marital_Status_Code']

X=df[df['Response']==1][col].values

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
```

Метод локтя.

```python
sse = {}
for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(X)
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Количество кластеров")
plt.ylabel("SSE")
plt.show()
```

Формирование таблицы с результатами.

```python
kmeans = KMeans(n_clusters = 5).fit(X)
y_kmeans = kmeans.predict(X)
df_result=df[df['Response']==1][col].copy()
df_result['kmeans']=kmeans.labels_
df_result.sample(10)
```

Групировка результатов вокруг кластеров. 

```python
df_analize=df_result.groupby('kmeans')[col].mean()
df_analize['Count']=df_result.groupby('kmeans')['kmeans'].count()
df_analize
```

Визуализация результатов.

```python
x=col[1] #Изменяйте столбцы 
y=col[2]
g = sns.lmplot(x=x, y=y, hue="kmeans", col="kmeans",
               data=df_result, height=6, aspect=.4, x_jitter=.1)
```

# Предсказание

## Классификация

Подготовка данных.


```python
from sklearn import preprocessing

Y=df['Response'].values
col2=['Year_Birth', 'Income', 'Recency', 'NumWebVisitsMonth', 'Complain', 
       'Total_Engaged_Days', 'Age', 'Marital_Status_Code', 'Education_Code']

X = preprocessing.MinMaxScaler().fit_transform(df[col2])

from sklearn.model_selection import train_test_split
#разделим набор на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
```

Классификация.

```python
from sklearn.ensemble import RandomForestClassifier
#feature extraction
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

from sklearn.metrics import r2_score
print(r2_score(model.predict(X_train), y_train))
print(r2_score(model.predict(X_test), y_test))
```

Матрица количества правильно и ошибочно угаданных классов

```python

from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, model.predict(X_test)), columns=[0,1], index=[0,1])
```

## Регрессия

Подготовка данных.

```python
Y=df['Income'].values
col2=['Year_Birth', 'Recency', 'NumWebVisitsMonth', 'Complain', 
       'Total_Engaged_Days', 'Age', 'Marital_Status_Code', 'Education_Code', 'Response']

X = preprocessing.MinMaxScaler().fit_transform(df[col2])

from sklearn.model_selection import train_test_split
#разделим набор на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
```

Выполняем регрессию.

```python
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

from sklearn.metrics import r2_score
print(r2_score(model.predict(X_train), y_train))
print(r2_score(model.predict(X_test), y_test))

plt.plot(model.predict(X_test), y_test, 'o');
```